In [ ]:
import numpy as np
from LSTM_Learning_Lib import Model
from FeatureSetCalculation_Lib import ComputeMultiLevelLogsig1dBM
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import ParameterGrid
from sklearn import preprocessing
import random

## Data loading

In [ ]:
BM_paths = np.load('BM_paths.npy')
number_of_samples = BM_paths.shape[0]
"""
BM1001 = np.zeros([number_of_samples, 1001])
for i in range(number_of_samples):
    for j in range(5001):
        if j%5==0:
            BM1001[i][int(j/5)] = BM_paths[i][j]
BM_paths = BM1001
"""
print(BM_paths.shape)
output = np.load('output.npy')
T=1

## Hyperparameters

In [ ]:
# Parameters grid
param_grid = {'deg_of_sig': [2,3,4,5,6], 'number_of_segment': [4],
'learning_rate': [0.001]}
Param = list(ParameterGrid(param_grid))
# Parameters
training_iters = 8000000
batch_size = 1024
display_step = 1
# Network Parameters
n_input = 1 # 
n_steps = np.shape(BM_paths)[1] # timesteps
n_hidden = 64 # hidden layer num of features
n_classes = 1 # the dim of the solution to SDE

Y = output
error_tol = 0.001/512

test_len = 200 #int(np.shape(BM_paths)[0]*0.1)
sig_comp_time = []
test_result = []
test_time = []

## train RNN with different feature sets

In [ ]:
for i in range(np.size(Param)):
    start = time.time()
    # Raw data feature set generator
    if Param[i]['deg_of_sig']==1:
        n_input = 1
        X_raw = BM_paths.reshape(-1, BM_paths.shape[1], n_input)
        print(X_raw.shape)
        model3 = Model( Param[i]['learning_rate'], training_iters, batch_size, display_step, n_input, X_raw.shape[1], n_hidden, n_classes, Param[i]['deg_of_sig'], X_raw, Y)
    # Folded raw data feature set generator
    elif Param[i]['deg_of_sig']==0:
        n_input = int(BM_paths.shape[1]/Param[i]['number_of_segment'])
        X_raw = np.zeros([BM_paths.shape[0], n_input * Param[i]['number_of_segment']])
        for j in range(BM_paths.shape[0]):
            X_raw[j] = BM_paths[j, :n_input * Param[i]['number_of_segment']]
        X_raw = X_raw.reshape(BM_paths.shape[0], Param[i]['number_of_segment'], n_input)
        elapsed = time.time()-start
        sig_comp_time.append(elapsed)
        model3 = Model( Param[i]['learning_rate'], training_iters, batch_size, display_step, n_input, Param[i]['number_of_segment'], n_hidden, n_classes, Param[i]['deg_of_sig'], X_raw, Y)
    # Logsig feature set generator
    else:
        X_logsig_start = ComputeMultiLevelLogsig1dBM(BM_paths, Param[i]['number_of_segment'], Param[i]['deg_of_sig'], T)
        n_input = np.shape(X_logsig_start)[2]
        elapsed = time.time()-start
        sig_comp_time.append(elapsed)
        model3 = Model( Param[i]['learning_rate'], training_iters, batch_size, display_step, n_input, Param[i]['number_of_segment'], n_hidden, n_classes, Param[i]['deg_of_sig'], X_logsig_start, Y)


    # build and train model
    fixed_error_result_model3 = model3.BuildModelKeras(test_len, error_tol)
    

    print("Time = " + str(time.time()-start))
    print("Testing loss = " + str(fixed_error_result_model3['Loss']))
    # model3.KerasPredict()
    test_result.append(fixed_error_result_model3 ['Loss'])
    test_time.append(fixed_error_result_model3 ['Time'])
    
    
    np.save('error_tol'+str(error_tol)+'deg_logsig'+str(Param[i]['deg_of_sig'])+'_test_result', test_result)
    np.save('error_tol'+str(error_tol)+'deg_logsig'+str(Param[i]['deg_of_sig'])+'_test_time', test_time)
    np.save('error_tol'+str(error_tol)+'deg_logsig'+str(Param[i]['deg_of_sig'])+'_sig_comp_time', elapsed)

In [ ]:
print(test_time)
print(test_result)
print(sig_comp_time)